In [1]:
from autogen_agentchat.agents import CodingAssistantAgent, AssistantAgent
from autogen_agentchat.task import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import AzureOpenAIChatCompletionClient

import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')

In [3]:
client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True,
    }
)

In [4]:
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed tfinal travel plan. You must ensure th b at the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

In [5]:
termination = TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent],
    termination_condition=termination
)

In [6]:
result = await group_chat.run(task="Plan a 3 day trip to Paris.")

In [7]:
from IPython.display import Markdown, display

total_prompt_tokens = 0
total_completion_tokens = 0

for index, message in enumerate(result.messages, start=1):
    display(Markdown('<hr>'))
    text = f"## Step: {index} (output from {message.source})"
    display(Markdown(text))

    if message.models_usage:
        print(f'Prompt Tokens: {message.models_usage.prompt_tokens}')
        print(f'Completion Tokens: {message.models_usage.completion_tokens}')
        total_prompt_tokens += message.models_usage.prompt_tokens
        total_completion_tokens += message.models_usage.completion_tokens
    display(Markdown('<hr>'))

    display(Markdown(message.content))

display(Markdown('<hr>'))
print(f'Total Prompt Tokens: {total_prompt_tokens}')
print(f'Total Completion Tokens: {total_completion_tokens}')
print(f'Total Tokens: {total_prompt_tokens + total_completion_tokens}')

<hr>

## Step: 1 (output from user)

<hr>

Plan a 3 day trip to Paris.

<hr>

...

Prompt Tokens: 1917
Completion Tokens: 726


<hr>

Here's a refined 3-day itinerary for your trip to Paris, integrating all the great suggestions from travel, local, and language perspectives. This plan is designed to help you experience both the iconic and local flavors of Paris.

### Day 1: Classic and Cultural Paris

**Morning:**
- **Eiffel Tower:** Start your journey with the iconic Eiffel Tower. Arrive early to enjoy the view from the top or admire it from the gardens.
  
**Lunch:**
- **Les Cocottes:** Savor a modern twist on traditional French cuisine at this nearby contemporary bistro.

**Afternoon:**
- **Musée d'Orsay:** Explore this stunning museum with its Impressionist masterpieces. It's less crowded than the Louvre and offers an enriching artistic experience.

**Evening:**
- **Saint-Germain-des-Prés:** Stroll through this historic area, enjoying an aperitif at famous cafes like **Café de Flore**.
- **Dinner:** Relish a classic steak-frites dinner at **Le Relais de l'Entrecôte**.

### Day 2: Artistic Exploration

**Morning:**
- **Louvre Museum:** Visit this world-renowned museum early to catch a glimpse of the Mona Lisa without large crowds. Pre-book your tickets to avoid delays.

**Lunch:**
- **Angelina Paris:** Treat yourself to decadent hot chocolate and pastries at this elegant tea room.

**Afternoon:**
- **Île Saint-Louis & Notre-Dame:** Discover the charming Île Saint-Louis and take a walk around the serene Notre-Dame exterior. Don’t forget a scoop of Berthillon ice cream.
- **Sainte-Chapelle:** Enter this nearby gothic chapel to marvel at its breathtaking stained-glass windows.

**Evening:**
- **Marais District:** Meander through trendy streets filled with shops and cafés.
- **Dinner:** Enjoy the Provencal flair at **Chez Janou** in the heart of the Marais.

### Day 3: Bohemian and Panoramic Paris

**Morning:**
- **Montmartre:** Begin with Sacré-Cœur Basilica’s panoramic views. Explore artistic lanes and appreciate the work at Place du Tertre.

**Lunch:**
- **La Maison Rose:** Dine at this quintessentially Parisian spot known for its scenic setting and delightful menu.

**Afternoon:**
- **Passey Village & Musée de Montmartre:** Discover Montmartre’s peaceful ambiance and reflect on its artistic history at this small museum.

**Evening:**
- **Seine River Dinner Cruise:** Conclude your adventure with a dinner cruise that offers stunning views of Paris illuminated at night.
- **Optional Dinner:** For an upscale experience, try **L'Atelier de Joël Robuchon** for innovative French cuisine.

### Tips for a Seamless Experience:

- **Accommodation:** Central areas like the 1st, 4th, and 7th arrondissements make for convenient stays.
- **Transportation:** Use a Paris Visite pass or rent a bicycle for authentic exploration.
- **Language Preparation:** Use basic French phrases such as "Bonjour," "Merci," and "Parlez-vous anglais?" for polite interactions. Smartphone apps can aid with translations if needed.
- **Local Interaction:** Engage with locals for tips, or simply enjoy the art of people-watching at parks like Jardin du Luxembourg.

Enjoy your delightful Parisian getaway, embracing both the unforgettable landmarks and the hidden gems that make the City of Light truly magical. Safe travels and immerse yourself in the beauty of Paris! 

**TERMINATE**

<hr>

Total Prompt Tokens: 4335
Total Completion Tokens: 2532
Total Tokens: 6867
